In [ ]:
%%capture
!pip install datasets
!pip install optuna

In [ ]:
# The value for the seed and paths
SEED = 168
PATH_TO_W2V_MODEL_DRIVE = '/content/drive/MyDrive/NLP 24-25 Sem1/GoogleNews-vectors-negative300.bin'
PATH_TO_W2V_MODEL_LOCAL = r'C:/Users/60178/Downloads/GoogleNews-vectors-negative300.bin'
PATH_TO_FASTTEXT_MODEL_DRIVE = '/content/drive/MyDrive/NLP 24-25 Sem1/cc.en.300.bin'
PATH_TO_FASTTEXT_MODEL_LOCAL = r'C:/Users/60178/Downloads/cc.en.300.bin/cc.en.300.bin'

In [ ]:
# Standard library imports
from collections import Counter

# General third-party libraries import
import copy
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_model
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import random
import string
import sys

# Import optuna for hyperparams tuning
import optuna

# Import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary nltk resources
nltk.download('wordnet')
nltk.download('punkt')  # Ensure punkt is downloaded for tokenization
nltk.download('stopwords')  # Ensure stopwords are also downloaded if not already

# Import pytorch
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn.utils.rnn import pad_sequence

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Method to set random seed to ensure consistency
def set_seed(seed = SEED):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# Factory to create the dataloader
def dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device):
    # Create Tensor datasets
    train_dataset = TensorDataset(X_train_sequence.to(device), torch.tensor(Y_train, dtype=torch.long, device=device))
    val_dataset = TensorDataset(X_val_sequence.to(device), torch.tensor(Y_val, dtype=torch.long, device=device))
    test_dataset = TensorDataset(X_test_sequence.to(device), torch.tensor(Y_test, dtype=torch.long, device=device))

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader

# Early Stopper
class EarlyStopper:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.max_validation_accuracy = 0

    def early_stop(self, validation_accuracy):
        if validation_accuracy > self.max_validation_accuracy:
            self.max_validation_accuracy = validation_accuracy
            self.counter = 0
        elif validation_accuracy < (self.max_validation_accuracy + self.delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

# Factory to provide required optimizer which specified parameters
def optimizer_factory(optimizer_name, model, learning_rate, momentum = 0.8):
    if optimizer_name == 'Adam':
        return torch.optim.Adam(model.parameters(), lr = learning_rate)
    if optimizer_name == 'SGD':
        return torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = momentum)
    if optimizer_name == 'RMSprop':
        return torch.optim.RMSprop(model.parameters(), lr = learning_rate)

# Method to train the model
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()  # Set model to training mode
    train_loss = 0  # To accumulate loss for this epoch

    for inputs, labels in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        # Calculate loss
        loss = loss_fn(outputs.squeeze(), labels.float())  # Squeeze to match dimensions
        loss.backward()  # Backpropagation

        # # Apply gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        optimizer.step()  # Update parameters

        train_loss += loss.item()  # Accumulate loss

    # Calculate average loss for the epoch
    train_loss /= len(dataloader)
    return train_loss

# Method to evaluate the model
def test_loop(dataloader, model):
    model.eval() # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in dataloader:
            outputs = model(inputs)  # Forward pass
            predicted = (outputs.squeeze() > 0.5).float()  # Apply threshold

            total += labels.size(0)
            correct += (predicted == labels.float()).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Method to plot the performance graph
def plot_performance(train_losses, val_accuracies, num_epochs):
    # Visualization
    plt.figure(figsize=(12, 5))

    # Plotting Loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, num_epochs + 1, 10))
    plt.legend()

    # Plotting Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.xticks(range(1, num_epochs + 1, 10))
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
set_seed()
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

# **Part 0. Dataset Preparation**

Part 0. Dataset Preparation We will be using the movie review dataset introduced in https://www.cs.cornell.edu/people/ pabo/movie-review-data/rt-polaritydata.README.1.0.txt. To load this dataset, you need to install the “datasets” library via pip install datasets. Then you can use the following code snippet: 1 from datasets import load_dataset 2 dataset = load_dataset (" rotten_tomatoes ") 3 train_dataset = dataset [’train ’] 4 validation_dataset = dataset [’validation ’] 5 test_dataset = dataset [’test ’] Using the original train-valid-test split provided in the above code, you will perform model training on the training dataset, configure your model (e.g., learning rate, batch size, number of training epochs) on the validation dataset, and conduct evaluation on the test dataset.

# **Load Dataset**

In [ ]:
set_seed()
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [ ]:
print(f"train_dataset size: {len(train_dataset)}")
print(f"validation_dataset size: {len(validation_dataset)}")
print(f"test_dataset size: {len(test_dataset)}")
print(f"data format: {train_dataset[0]}")

train_dataset size: 8530
validation_dataset size: 1066
test_dataset size: 1066
data format: {'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}


In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"The name of the CUDA device is: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA is available!
The name of the CUDA device is: Tesla P100-PCIE-16GB


# **Load word2vec Model**

In [ ]:
# Load the pre-trained Word2Vec model
if 'google.colab' in sys.modules:
    print("Running on Google Colab's hosted runtime")
    from google.colab import drive
    drive.mount('/content/drive')
    w2v_model = KeyedVectors.load_word2vec_format(PATH_TO_W2V_MODEL_DRIVE, binary=True)
else:
    print("Running on a local runtime")
    w2v_model = KeyedVectors.load_word2vec_format(PATH_TO_W2V_MODEL_LOCAL, binary=True)

# Get the dimension of the embeddings
vector_dim = w2v_model.vector_size
print(f"Word2Vec Dimension: {vector_dim}")


Running on a local runtime
Word2Vec Dimension: 300


# **Data Preprocessing**

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [ ]:
# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))

# Dataset Preprocessing to tokenize and/or lemmatize
def preprocess_dataset(data, lemmatization = True):
    processed_sentences = []

    for entry in data:
        text = entry['text']
        # Tokenize the sentence
        tokens = word_tokenize(text)
        if lemmatization:
          # With case folding, punctuation and stop words removal
          #tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word not in punctuation and word.lower() not in stop_words]
          tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word not in punctuation]

        processed_sentences.append(tokens)

    return processed_sentences

# Get vocab and oov words
def get_vocab_OOV(sentences, w2v_model):
    vocabulary = set()
    oov_words = set()

    for sentence in sentences:
        for word in sentence:
            vocabulary.add(word)
            if word not in w2v_model:
                oov_words.add(word)

    return vocabulary, oov_words

vocabulary, oov_words = get_vocab_OOV(preprocess_dataset(train_dataset, False), w2v_model)
lemmatized_vocabulary, lemmatized_oov_words = get_vocab_OOV(preprocess_dataset(train_dataset), w2v_model)

print(f"Before lemmatization: Vocabulary size: {len(vocabulary)}, OOV words: {len(oov_words)}")
print(f"After lemmatization: Vocabulary size: {len(lemmatized_vocabulary)}, OOV words: {len(lemmatized_oov_words)}")

Before lemmatization: Vocabulary size: 18035, OOV words: 3619
After lemmatization: Vocabulary size: 16571, OOV words: 3590




1.   Lemmatize, case folding and remove stop words for train, validation and test set
2.   Drop words not found in train dataset for validation and test set




In [ ]:
preprocessed_train_dataset = preprocess_dataset(train_dataset)
preprocessed_validation_dataset = preprocess_dataset(validation_dataset)
preprocessed_test_dataset = preprocess_dataset(test_dataset)

In [ ]:
def drop_oov(preprocessed_dataset, w2v_model):
    return [[word for word in sentence if word in w2v_model] for sentence in preprocessed_dataset]

def drop_non_train_vocab(dataset, train_vocab):
    return [[word for word in sentence if word in train_vocab] for sentence in dataset]

X_train = preprocessed_train_dataset
Y_train = [entry['label'] for entry in train_dataset]

X_val = drop_non_train_vocab(preprocessed_validation_dataset, lemmatized_vocabulary)
Y_val = [entry['label'] for entry in validation_dataset]

X_test = drop_non_train_vocab(preprocessed_test_dataset, lemmatized_vocabulary)
Y_test = [entry['label'] for entry in test_dataset]


Sanity check: If preprocessed data is wrong

In [ ]:
print(f"X_train size: {len(X_train)}")
print(f"Y_train size: {len(Y_train)}")

print(f"X_val size: {len(X_val)}")
print(f"Y_val size: {len(Y_val)}")

print(f"X_test size: {len(X_test)}")
print(f"Y_test size: {len(Y_test)}")

X_train size: 8530
Y_train size: 8530
X_val size: 1066
Y_val size: 1066
X_test size: 1066
Y_test size: 1066


# **Generate Embedding**

In [ ]:
def get_word2index(train_vocab):
    word2index = {}
    for idx, word in enumerate(train_vocab):
        word2index[word] = idx + 1  # +1 to offset 0 for padding
    return word2index

word2index = get_word2index(lemmatized_vocabulary)

In [ ]:
print(f"word2index size = vocab size: {len(word2index)}")

word2index size = vocab size: 16571


In [ ]:
def create_embedding_matrix(w2v_model, word2index, embedding_dim=300):
    """
    Creates an embedding matrix based on the word2index mapping and Word2Vec model.

    Args:
    - w2v_model: Pre-trained Word2Vec model.
    - word2index: Dictionary mapping words to their indices.
    - embedding_dim: Dimension of the Word2Vec word vectors (default: 300).

    Returns:
    - embedding_matrix: Embedding matrix where each row corresponds to the vector of a word in the vocabulary.
    """
    vocab_size = len(word2index)  # Number of words in the training vocabulary

    embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))
    mean = np.mean(w2v_model.vectors, axis=0)
    std = np.std(w2v_model.vectors, axis=0)

    for word, idx in word2index.items():
        if word in w2v_model:
            embedding_matrix[idx] = w2v_model[word]
        else:
            embedding_matrix[idx] = np.random.normal(
                loc=mean, scale=std, size=(embedding_dim,)
            )

    return embedding_matrix

embedding_dim = w2v_model.vector_size  # Get the dimension of the word vectors in Word2Vec
embedding_matrix = create_embedding_matrix(w2v_model, word2index, embedding_dim)
print(f"embedding_matrix shape: {embedding_matrix.shape}")


embedding_matrix shape: (16572, 300)


# **Sentence2sequence + Padding**

In [ ]:
# Get maximum sentence length
def get_max_sentence_length(*datasets):
    """
    Get the maximum sentence length from the provided datasets.

    Args:
    - datasets: A variable number of lists containing tokenized sentences.

    Returns:
    - max_length: The maximum sentence length across all datasets.
    """
    max_length = 0
    for index, dataset in enumerate(datasets):
        # Update max_length if a longer sentence is found
        max_length = max(max_length, max(len(sentence) for sentence in dataset))
        min_length = min(len(sentence) for sentence in dataset)
        print(f"Min sentence length in dataset {index}: {min_length}")
    return max_length

# Example usage
max_length = get_max_sentence_length(X_train, X_val, X_test)
print(f"Maximum sentence length of three datasets: {max_length}")

Min sentence length in dataset 0: 1
Min sentence length in dataset 1: 0
Min sentence length in dataset 2: 1
Maximum sentence length of three datasets: 53


In [ ]:
def convert_to_sequence_data(tokenized_data, word2index, max_length=max_length):
    """
    Preprocess the tokenized data for RNN input.

    Args:
    - tokenized_data: List of tokenized sentences.
    - word2index: Dictionary mapping words to indices.
    - max_length: Maximum length for padding (default is 100).

    Returns:
    - padded_sequences: Padded tensor of sequences.
    """
    # Convert tokens to indices
    indexed_sequences = []
    for sentence in tokenized_data:
        indexed_sentence = [word2index.get(word, 0) for word in sentence]  # Use 0 for OOV words

        # Truncate or pad the indexed sentence to the specified max_length
        if len(indexed_sentence) > max_length:
            indexed_sentence = indexed_sentence[:max_length]  # Truncate
        else:
            indexed_sentence += [0] * (max_length - len(indexed_sentence))  # Pad with zeros

        indexed_sequences.append(torch.tensor(indexed_sentence, dtype=torch.long))

    # Convert list of tensors to a padded tensor
    padded_sequences = torch.stack(indexed_sequences)

    return padded_sequences

# Convert to sequence
X_train_sequence = convert_to_sequence_data(X_train, word2index)
X_val_sequence = convert_to_sequence_data(X_val , word2index)
X_test_sequence = convert_to_sequence_data(X_test, word2index)


Sanity check: if the sequenced tokens mapped correctly to embedding matrix and w2v_model

In [ ]:
# Check if the sequenced tokens mapped correctly to embedding matrix and w2v_model
embedding_matrix[X_train_sequence[0][0]] == w2v_model[X_train[0][0]]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
print(f"Shape of X_train_sequence: {X_train_sequence.shape}")
print(f"Shape of X_valn_sequence: {X_val_sequence.shape}")
print(f"Shape of X_test_sequence: {X_test_sequence.shape}")

Shape of X_train_sequence: torch.Size([8530, 53])
Shape of X_valn_sequence: torch.Size([1066, 53])
Shape of X_test_sequence: torch.Size([1066, 53])


## **Training**

# **CNN Model Architecture**

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, embedding_matrix, num_filters=64, num_conv_layers=1, filter_size=3, aggregation='mean_pool', attn = False, num_heads=1):
        super(SimpleCNN, self).__init__()
        embedding_dim = embedding_matrix.shape[1]
        self.aggregation = aggregation
        self.attn = attn

        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            padding_idx=0,  # Assuming 0 is the padding index
            freeze=False  # Unfreeze the embeddings
        )

        self.conv1d_layers = nn.ModuleList([
            nn.Sequential(*[
                nn.Conv1d(in_channels=(embedding_dim if i == 0 else num_filters),  # Use embedding_dim for the first layer
                          out_channels=num_filters,
                          kernel_size=filter_size),
                nn.ReLU(),
            ]) for i in range(num_conv_layers)
        ])
        if self.attn:
            self.attn_head = nn.MultiheadAttention(num_filters, num_heads, batch_first = True)

        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(num_filters, 1)  # Output size is 1 for binary classification

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.permute(0, 2, 1)

        conv_out = embedded
        for conv1d_layer in self.conv1d_layers:
            conv_out = conv1d_layer(conv_out)

        if self.aggregation == 'max_pool':
            agg_out, _ = torch.max(conv_out, dim=2)  # Max-pooling over the sequence length
        elif self.aggregation == 'mean_pool':
            agg_out = torch.mean(conv_out, dim=2)  # Mean-pooling over the sequence length
        if self.attn:
            agg_out, _ = self.attn_head(agg_out, agg_out, agg_out) # Adding attention head after aggregation
        out = self.dropout(agg_out)
        out = self.fc(out)
        out = torch.sigmoid(out)

        return out

# **CNN Hyperparameter Tuning**

In [ ]:
def cnn_objective(trial):

    max_accuracy=0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Reset the seed to ensure fairness when comparing performances

    # Hyperparameters
    num_filters = trial.suggest_categorical('num_filters', [64, 128, 256])  # Hidden size choices
    num_conv_layers = trial.suggest_categorical('num_conv_layers', [1, 2, 3])  # Number of RNN layers choices
    filter_size = trial.suggest_categorical('filter_size', [3, 5, 7])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 5e-4, 1e-4, 1e-3])  # Learning rate choices
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size choices
    num_epochs = trial.suggest_categorical('num_epochs', [10, 20, 30, 40, 50])  # num epoch choices
    # aggregation = trial.suggest_categorical('aggregation', ['max_pool', 'mean_pool'])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])  # Optimizer choices


    # Create data loaders
    train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

    # Initialize the model
    model = SimpleCNN(embedding_matrix, num_filters, num_conv_layers, filter_size, 'max_pool', False, 1)
    model.to(device)
    criterion = nn.BCELoss()  # Use BCELoss for binary classification

    # Choose optimizer based on trial suggestion
    optimizer = optimizer_factory(optimizer_name, model, learning_rate)

    # Lists to store loss and accuracy for visualization
    train_losses = []
    val_accuracies = []

    # Training Loop
    for epoch in range(num_epochs):
        loss = train_loop(train_loader, model, criterion, optimizer)
        train_losses.append(loss)

        # Evaluate on validation set
        val_accuracy = test_loop(val_loader, model)
        val_accuracies.append(val_accuracy)
        if val_accuracy>max_accuracy:
          max_accuracy = val_accuracy

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    return max_accuracy  # Return validation accuracy for optimization

set_seed()
study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(cnn_objective, n_trials=100)  # Invoke optimization of the objective function.

[I 2024-10-31 04:42:17,896] A new study created in memory with name: no-name-e3250d23-beeb-40b7-bcad-709fdfce7732


Epoch [1/30], Train Loss: 0.6927, Validation Accuracy: 0.5300
Epoch [2/30], Train Loss: 0.6825, Validation Accuracy: 0.6689
Epoch [3/30], Train Loss: 0.6463, Validation Accuracy: 0.7364
Epoch [4/30], Train Loss: 0.5591, Validation Accuracy: 0.7655
Epoch [5/30], Train Loss: 0.4732, Validation Accuracy: 0.7570
Epoch [6/30], Train Loss: 0.4200, Validation Accuracy: 0.7636
Epoch [7/30], Train Loss: 0.3778, Validation Accuracy: 0.7617
Epoch [8/30], Train Loss: 0.3470, Validation Accuracy: 0.7692
Epoch [9/30], Train Loss: 0.3170, Validation Accuracy: 0.7589
Epoch [10/30], Train Loss: 0.2868, Validation Accuracy: 0.7617
Epoch [11/30], Train Loss: 0.2588, Validation Accuracy: 0.7702
Epoch [12/30], Train Loss: 0.2320, Validation Accuracy: 0.7720
Epoch [13/30], Train Loss: 0.2082, Validation Accuracy: 0.7655
Epoch [14/30], Train Loss: 0.1880, Validation Accuracy: 0.7636
Epoch [15/30], Train Loss: 0.1686, Validation Accuracy: 0.7645
Epoch [16/30], Train Loss: 0.1511, Validation Accuracy: 0.7598
E

[I 2024-10-31 04:42:35,325] Trial 0 finished with value: 0.7720450281425891 and parameters: {'num_filters': 64, 'num_conv_layers': 2, 'filter_size': 3, 'learning_rate': 0.0001, 'batch_size': 128, 'num_epochs': 30, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7720450281425891.


Epoch [30/30], Train Loss: 0.0327, Validation Accuracy: 0.7505
Epoch [1/40], Train Loss: 0.6940, Validation Accuracy: 0.5075
Epoch [2/40], Train Loss: 0.6916, Validation Accuracy: 0.5657
Epoch [3/40], Train Loss: 0.6901, Validation Accuracy: 0.5994
Epoch [4/40], Train Loss: 0.6886, Validation Accuracy: 0.6154
Epoch [5/40], Train Loss: 0.6873, Validation Accuracy: 0.6379
Epoch [6/40], Train Loss: 0.6857, Validation Accuracy: 0.6510
Epoch [7/40], Train Loss: 0.6843, Validation Accuracy: 0.6510
Epoch [8/40], Train Loss: 0.6826, Validation Accuracy: 0.6707
Epoch [9/40], Train Loss: 0.6805, Validation Accuracy: 0.6698
Epoch [10/40], Train Loss: 0.6791, Validation Accuracy: 0.6735
Epoch [11/40], Train Loss: 0.6767, Validation Accuracy: 0.6698
Epoch [12/40], Train Loss: 0.6749, Validation Accuracy: 0.6829
Epoch [13/40], Train Loss: 0.6724, Validation Accuracy: 0.6801
Epoch [14/40], Train Loss: 0.6701, Validation Accuracy: 0.6876
Epoch [15/40], Train Loss: 0.6680, Validation Accuracy: 0.6923
E

[I 2024-10-31 04:42:57,485] Trial 1 finished with value: 0.7382739212007504 and parameters: {'num_filters': 256, 'num_conv_layers': 1, 'filter_size': 7, 'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 40, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.7720450281425891.


...

Epoch [50/50], Train Loss: 0.0000, Validation Accuracy: 0.7505
Epoch [1/40], Train Loss: 0.6932, Validation Accuracy: 0.5047
Epoch [2/40], Train Loss: 0.6920, Validation Accuracy: 0.5375
Epoch [3/40], Train Loss: 0.6899, Validation Accuracy: 0.5966
Epoch [4/40], Train Loss: 0.6885, Validation Accuracy: 0.6567
Epoch [5/40], Train Loss: 0.6866, Validation Accuracy: 0.6867
Epoch [6/40], Train Loss: 0.6854, Validation Accuracy: 0.6904
Epoch [7/40], Train Loss: 0.6837, Validation Accuracy: 0.6998
Epoch [8/40], Train Loss: 0.6822, Validation Accuracy: 0.7054
Epoch [9/40], Train Loss: 0.6798, Validation Accuracy: 0.7054
Epoch [10/40], Train Loss: 0.6781, Validation Accuracy: 0.7054
Epoch [11/40], Train Loss: 0.6764, Validation Accuracy: 0.7111
Epoch [12/40], Train Loss: 0.6741, Validation Accuracy: 0.7120
Epoch [13/40], Train Loss: 0.6713, Validation Accuracy: 0.7101
Epoch [14/40], Train Loss: 0.6696, Validation Accuracy: 0.7083
Epoch [15/40], Train Loss: 0.6666, Validation Accuracy: 0.7054
E

[I 2024-10-31 05:29:17,253] Trial 98 finished with value: 0.7514071294559099 and parameters: {'num_filters': 64, 'num_conv_layers': 1, 'filter_size': 7, 'learning_rate': 0.0005, 'batch_size': 32, 'num_epochs': 40, 'optimizer': 'SGD'}. Best is trial 34 with value: 0.8020637898686679.


Epoch [40/40], Train Loss: 0.5401, Validation Accuracy: 0.7505
Epoch [1/50], Train Loss: 0.6929, Validation Accuracy: 0.5760
Epoch [2/50], Train Loss: 0.6880, Validation Accuracy: 0.6773
Epoch [3/50], Train Loss: 0.6202, Validation Accuracy: 0.7083
Epoch [4/50], Train Loss: 0.5332, Validation Accuracy: 0.7392
Epoch [5/50], Train Loss: 0.4996, Validation Accuracy: 0.7448
Epoch [6/50], Train Loss: 0.4781, Validation Accuracy: 0.7580
Epoch [7/50], Train Loss: 0.4610, Validation Accuracy: 0.7636
Epoch [8/50], Train Loss: 0.4458, Validation Accuracy: 0.7636
Epoch [9/50], Train Loss: 0.4305, Validation Accuracy: 0.7702
Epoch [10/50], Train Loss: 0.4163, Validation Accuracy: 0.7636
Epoch [11/50], Train Loss: 0.4042, Validation Accuracy: 0.7692
Epoch [12/50], Train Loss: 0.3900, Validation Accuracy: 0.7711
Epoch [13/50], Train Loss: 0.3780, Validation Accuracy: 0.7627
Epoch [14/50], Train Loss: 0.3636, Validation Accuracy: 0.7655
Epoch [15/50], Train Loss: 0.3519, Validation Accuracy: 0.7702
E

[I 2024-10-31 05:30:26,301] Trial 99 finished with value: 0.7823639774859287 and parameters: {'num_filters': 256, 'num_conv_layers': 3, 'filter_size': 7, 'learning_rate': 1e-05, 'batch_size': 32, 'num_epochs': 50, 'optimizer': 'Adam'}. Best is trial 34 with value: 0.8020637898686679.


Epoch [50/50], Train Loss: 0.0199, Validation Accuracy: 0.7580
